In [24]:
from dotenv import load_dotenv

load_dotenv()

import streamlit as st
import pandas as pd

from pydantic import BaseModel, Field
from typing import Optional

from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.agents import create_pandas_dataframe_agent
from typing import List
from pydantic import BaseModel
import utils
import concurrent.futures

from typing import List, Optional

from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI

import pandas as pd
from pydantic import BaseModel, Field, validator

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import re

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


st.set_page_config(layout="wide")

class ContactInfo:
    def __init__(self, first_name, last_name, job_title, company_name, mobile_number, office_number, business_website, email, address):
        self.first_name = first_name
        self.last_name = last_name
        self.job_title = job_title
        self.company_name = company_name
        self.mobile_number = mobile_number
        self.office_number = office_number
        self.business_website = business_website
        self.email = email
        self.address = address

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def to_lowercase(df):
    return df.applymap(lambda s: s.lower() if type(s) == str else s)

def get_unique_contacts_with_most_info(df):
    # Add a column that counts the number of non-null fields for each contact
    df['info_count'] = df.notnull().sum(axis=1)
    # Group by 'first_name', 'last_name', 'email', and get the row with max 'info_count' for each group
    df = df.loc[df.groupby(['first_name', 'last_name', ])['info_count'].idxmax()]
    # Drop 'info_count' as it's no longer needed
    df = df.drop(columns='info_count')
    return df

def split_text_by_from(text):
    # Split the text by "From:" and strip any leading/trailing spaces from each part
    split_text = [part.strip() for part in text.split("From:")]

    # Remove the empty string at the beginning caused by the split
    split_text = [part for part in split_text if part]

    return split_text

def process_email(chain, email):
    output = chain.run(email)
    return pd.DataFrame(output.dict()['personal_info'])

def create_prompt(email):
    prompt = f"""

        System: As a world-class algorithm for extracting information in structured formats, your task is to extract the contact details from the given email's signature and cc section. The email is provided below:

        Human:
        {email}

        Human:
        Extract the contact information from the email signature and cc section.

        The information should be accurately extracted and arranged in a way that follows the correct format:

        Person 1
        First Name: 
        Last Name:
        Job Title:
        Email:
        Mobile Phone Number: 
        Office Phone Number: 
        Company Name: 
        Company Website:
        Company Address:

        Person 2
        First Name: 
        Last Name:
        Job Title:
        Email:
        Mobile Phone Number: 
        Office Phone Number: 
        Company Name: 
        Company Website:
        Company Address:

        Person N
        First Name: 
        Last Name:
        Job Title:
        Email:
        Mobile Phone Number: 
        Office Phone Number: 
        Company Name: 
        Company Website:
        Company Address:

        """
    return prompt

def parse_contact_info(input_list):
    personal_info_list = []
    entry_pattern = re.compile(
        r'\s*First Name:\s*(?P<first_name>[^\n]*)\s*'
        r'Last Name:\s*(?P<last_name>[^\n]*)\s*'
        r'Job Title:\s*(?P<job_title>[^\n]*)\s*'
        r'Email:\s*(?P<email>[^\n]*)\s*'
        r'Mobile Phone Number:\s*(?P<mobile_number>[^\n]*)\s*'
        r'Office Phone Number:\s*(?P<office_number>[^\n]*)\s*'
        r'Company Name:\s*(?P<company_name>[^\n]*)\s*'
        r'Company Website:\s*(?P<business_website>[^\n]*)\s*'
        r'Company Address:\s*(?P<address>[^\n]*)'
    )

    for entry in input_list:
        # strip leading/trailing whitespaces from the entry string
        entry = entry.strip()
        match = entry_pattern.search(entry)
        if match:
            contact_info = ContactInfo(
                first_name=match.group('first_name'),
                last_name=match.group('last_name'),
                job_title=match.group('job_title'),
                company_name=match.group('company_name'),
                mobile_number=match.group('mobile_number'),
                office_number=match.group('office_number'),
                business_website=match.group('business_website'),
                email=match.group('email'),
                address=match.group('address'),
            )
            personal_info_list.append(contact_info)

    return personal_info_list

def process_email(email):
    prompt = create_prompt(email)
    output = llm.predict(prompt)
    input_list = output.split('Person')
    result = parse_contact_info(input_list)
    found_contacts = pd.DataFrame([i.__dict__ for i in result])
    return found_contacts

In [25]:
conversation_input = """
From: Martin M. Zoltick <Mzoltick@rothwellfigg.com> 
Sent: Tuesday, July 25, 2023 4:21 PM
To: Nick Hergott <nhergott@surepoint.com>
Cc: Caroline B. O'Banion <cobanion@rothwellfigg.com>; Jennifer Whittier <jwhittier@surepoint.com>
Subject: RE: ContactEase Offer Update

***CAUTION:This email originated from outside SurePoint. Please exercise extreme caution with opening any attachments or links***
OK – understood – let’s leave the 3 yr term as originally discussed.
 
Just let me know on limit of liab and we should be all set.
 
THX
 
 
 
Marty Zoltick (he/him) 
Office: 202-626-3563
Mobile: 202-550-3563
Email:     mzoltick@rfem.com
Web:       www.rfem.com
 
    
 Be GREEN, keep it on the SCREEN
 
STATEMENT OF CONFIDENTIALITY
The information contained in this electronic message and any attachments to this message are intended for the exclusive use of the addressee(s) and may contain confidential and/or privileged information. If you are not the addressee, note that any disclosure, copy, distribution or use of the contents of this message is prohibited. If you are not the intended recipient, please notify Rothwell, Figg, Ernst & Manbeck, P.C. immediately at (202) 783-6040 or email us at mzoltick@rfem.com, and destroy all copies of this message and any attachments.
 
From: Nick Hergott <nhergott@surepoint.com> 
Sent: Tuesday, July 25, 2023 3:24 PM
To: Martin M. Zoltick <Mzoltick@rothwellfigg.com>
Cc: Caroline B. O'Banion <cobanion@rothwellfigg.com>; Jennifer Whittier <jwhittier@surepoint.com>
Subject: [EXTERNAL] RE: ContactEase Offer Update
 
Hi Marty,
 
We’d be back up to $1,700/month with a 7% increase cap for a two-year option.  Our pricing contact was very clear that those concessions were made contingent on the three-year term.  However, I believe we may be able to build in 5% increases thereafter, if you’d prefer to go the two-year option.
 
I will pass along the note re: section 12 to our counsel for his response.  Please let me know if you’d prefer a contract for three years at $1,611 with 5% increases or a two-year option at $1,700 with a 7% increase, then an option to renew at 5%.
 
Thanks,
 
Nick Hergott
Sr. Account Executive
 
612-720-5193
surepoint.com
 
 


"""

In [26]:

emails = ['From:' + i for i in conversation_input.split('From:') if len(i) > 10]

main = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_email = {executor.submit(process_email, email): email for email in emails}
    from tqdm import tqdm
    for future in tqdm(concurrent.futures.as_completed(future_to_email)):
        email = future_to_email[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (email, exc))
        else:
            print('%r extracted successfully' % (email))
            main.append(data)

1it [00:07,  7.26s/it]

"From: Martin M. Zoltick <Mzoltick@rothwellfigg.com> \nSent: Tuesday, July 25, 2023 4:21 PM\nTo: Nick Hergott <nhergott@surepoint.com>\nCc: Caroline B. O'Banion <cobanion@rothwellfigg.com>; Jennifer Whittier <jwhittier@surepoint.com>\nSubject: RE: ContactEase Offer Update\n\n***CAUTION:This email originated from outside SurePoint. Please exercise extreme caution with opening any attachments or links***\nOK – understood – let’s leave the 3 yr term as originally discussed.\n \nJust let me know on limit of liab and we should be all set.\n \nTHX\n \n \n \nMarty Zoltick (he/him) \nOffice: 202-626-3563\nMobile: 202-550-3563\nEmail:     mzoltick@rfem.com\nWeb:       www.rfem.com\n \n    \n\uf0fc Be GREEN, keep it on the SCREEN\n \nSTATEMENT OF CONFIDENTIALITY\nThe information contained in this electronic message and any attachments to this message are intended for the exclusive use of the addressee(s) and may contain confidential and/or privileged information. If you are not the addressee, no

2it [00:08,  4.40s/it]

"From: Nick Hergott <nhergott@surepoint.com> \nSent: Tuesday, July 25, 2023 3:24 PM\nTo: Martin M. Zoltick <Mzoltick@rothwellfigg.com>\nCc: Caroline B. O'Banion <cobanion@rothwellfigg.com>; Jennifer Whittier <jwhittier@surepoint.com>\nSubject: [EXTERNAL] RE: ContactEase Offer Update\n \nHi Marty,\n \nWe’d be back up to $1,700/month with a 7% increase cap for a two-year option.  Our pricing contact was very clear that those concessions were made contingent on the three-year term.  However, I believe we may be able to build in 5% increases thereafter, if you’d prefer to go the two-year option.\n \nI will pass along the note re: section 12 to our counsel for his response.  Please let me know if you’d prefer a contract for three years at $1,611 with 5% increases or a two-year option at $1,700 with a 7% increase, then an option to renew at 5%.\n \nThanks,\n \nNick Hergott\nSr. Account Executive\n \n612-720-5193\nsurepoint.com\n \n \n\n\n" extracted successfully


In [27]:
output = to_lowercase(pd.concat(main, ignore_index=True).drop_duplicates().replace('', None))

In [29]:
deduplicated_df = get_unique_contacts_with_most_info(output)

In [31]:
output = to_lowercase(pd.concat(main, ignore_index=True).drop_duplicates().replace('', None))

deduplicated_df = get_unique_contacts_with_most_info(output)

# Calculate the number of NaNs in each row
nan_counts = deduplicated_df.isna().sum(axis=1)

# Sort the DataFrame based on the least amount of NaNs
contacts = deduplicated_df.iloc[nan_counts.argsort()].to_dict('records')


In [32]:
contacts

[{'first_name': 'martin',
  'last_name': 'zoltick',
  'job_title': None,
  'company_name': 'rothwell, figg, ernst & manbeck, p.c.',
  'mobile_number': '202-550-3563',
  'office_number': '202-626-3563',
  'business_website': 'www.rfem.com',
  'email': 'mzoltick@rfem.com',
  'address': None},
 {'first_name': 'nick',
  'last_name': 'hergott',
  'job_title': 'sr. account executive',
  'company_name': 'surepoint',
  'mobile_number': '612-720-5193',
  'office_number': None,
  'business_website': 'surepoint.com',
  'email': 'nhergott@surepoint.com',
  'address': None},
 {'first_name': 'caroline',
  'last_name': "o'banion",
  'job_title': None,
  'company_name': 'rothwell, figg, ernst & manbeck, p.c.',
  'mobile_number': None,
  'office_number': None,
  'business_website': None,
  'email': 'cobanion@rothwellfigg.com',
  'address': None},
 {'first_name': 'jennifer',
  'last_name': 'whittier',
  'job_title': None,
  'company_name': 'surepoint',
  'mobile_number': None,
  'office_number': None,
 